##### 问题:
我们想调用一个父类中的方法，这个方法在子类中已经被覆盖了。

##### 解决方案:
要调用父类（或称超类）中的方法，可以使用 super()函数完成。示例如下：

In [80]:
class A:
    def spam(self):
        print('A.spam')
class B(A):
 def spam(self):
    print('B.spam')
    super().spam() # Call parent spam()


super()函数的一种常见用途是调用父类的__init__()方法，确保父类被正确地初始化了：

In [81]:
class A:
    def __init__(self):
        self.x = 0
class B(A):
 def __init__(self):
    super().__init__()
    self.y = 1

另一种常见用途是当覆盖了 Python 中的特殊方法时，示例如下：

In [82]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)
 # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value) # Call original __setattr__
        else:
            setattr(self._obj, name, value)

在上述代码中，\_\_setattr\_\_()的实现里包含了对名称的检查。如果名称是以一个下划线
（_）开头的，它就通过 super()去调用原始的__setattr__()实现。否则，就转而对内部持
有的对象 self._obj 进行操作。这看起来有点意思，但是 super()即使在没有显式列出基
类的情况下也是可以工作的。

如何正确使用 super()函数，这实际上是人们在 Python 中理解的最差的知识点之一。偶
尔我们会看到一些代码直接调用父类中的方法，就像这样：

In [83]:
class Base:
    def __init__(self):
        print('Base.__init__')
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

尽管对于大部分代码来说这么做都“行得通”，但是在涉及多重继承的代码里，就会导
致出现奇怪的麻烦。比如，考虑下面这个例子：

In [84]:
class Base:
    def __init__(self):
        print('Base.__init__')
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')
class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')
class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

如果运行上面的代码，会发现 Base.\_\_init\_\_()方法被调用了两次。如下所示：

In [85]:
c = C() 


Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


也许调用两次 Base.\_\_init\_\_()并没什么害处，但是也可能刚好相反。如果从另一方面考
虑，将代码修改为使用 super()，那么一切就都能正常工作了：

In [86]:
class Base:
    def __init__(self):
        print('Base.__init__')
class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')
class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
class C(A,B):
    def __init__(self):
        super().__init__() # Only one call to super() here
        print('C.__init__') 

当使用这个新版的代码时，就会发现每个__init__()方法都只调用了一次：

In [87]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


要理解其中的缘由，我们需要退一步，先讨论一下 Python 是如何实现继承的。针对每
一个定义的类，Python 都会计算出一个称为方法解析顺序（MRO）的列表①
。MRO 列
表只是简单地对所有的基类进行线性排列。示例如下：

In [88]:
C.__mro__


(__main__.C, __main__.A, __main__.B, __main__.Base, object)

要实现继承，Python 从 MRO 列表中最左边的类开始，从左到右依次查找，直到找到待
查的属性时为止。


而 MRO 列表本身又是如何确定的呢？这里用到了一种称为 C3 线性化处理（C3
Linearization）的技术。为了不陷入到艰深的数学理论中，简单来说这就是针对父类的
一种归并排序，它需要满足 3 个约束：


<li>先检查子类再检查父类；</li>
<li>有多个父类时，按照 MRO 列表的顺序依次检查；</li>
<li>如果下一个待选的类出现了两个合法的选择，那么就从第一个父类中选取。</li>

老实说，所有需要的知道的就是 MRO 列表中对类的排序几乎适用于任何定义的类层次
结构（class hierarchy）。

当使用 super()函数时，Python 会继续从 MRO 中的下一个类开始搜索。只要每一个重
新定义过的方法（也就是覆盖方法）都使用了 super()，并且只调用了它一次，那么控
制流最终就可以遍历整个 MRO 列表，并且让每个方法只会被调用一次。这就是为什
么在第二个例子中 Base.__init__()不会被调用两次的原因。

关于 super()，一个有些令人惊讶的方面是，它并不是一定要关联到某个类的直接父类
上，甚至可以在没有直接父类的类中使用它。例如，考虑下面这个类：

In [89]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

如果试着使用这个类，会发现这完全行不通：

In [90]:
a = A()
a.spam()

A.spam


AttributeError: 'super' object has no attribute 'spam'

但是，如果把这个类用于多重继承时看看会发生什么：

In [ ]:
class B:
    def spam(self):
        print('B.spam')

class C(A,B):
    pass

c = C()
c.spam()

A.spam
B.spam


这里我们会发现在类 A 中使用的 super().spam()实际上居然调用到了类 B 中的 spam()
方法——B 和 A 是完全不相关的！这一切都可以用类 C 的 MRO 列表来解释：

In [ ]:
C.__mro__


(__main__.C, __main__.A, __main__.B, __main__.Base, object)

我们常常会在定义混合类（mixin class）时以这种方式使用 super()

但是，由于 super()可能会调用到我们不希望调用的方法，那么这里有一些应该遵守的
基本准则。首先，确保在继承体系中所有同名的方法都有可兼容的调用签名（即，参
数数量相同，参数名称也相同）。如果 super()尝试去调用非直接父类的方法，那么这就
可以确保不会遇到麻烦。其次，确保最顶层的类实现了这个方法通常是个好主意。这
样沿着 MRO 列表展开的查询链会因为最终找到了实际的方法而终止。

在 Python 社区中，关于 super()的使用有时候会成为争论的焦点。但是，公平地说，我
们应该在现代的代码中使用它。Raymond Hettinger 在博客中写过一篇题为“Python’s
super() considered Super!”的文章，文章中列举了更多的示例和理由来说明为什么 super()
会是超级有用的工具。